In [ ]:
import pandas as pd

# Read Excel files
solutions_df = pd.read_excel("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")
categorical_df = pd.read_excel("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
quantitative_df = pd.read_excel("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")

# Read CSV file
fnc_df = pd.read_csv("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")


In [ ]:
quantitative_df

In [ ]:
quantitative_df.set_index('participant_id', inplace=True)
quantitative_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Set up the matplotlib figure
plt.figure(figsize=(20, 15))

# Loop through each column and plot its distribution, dropping NaNs for each column
for i, column in enumerate(quantitative_df.columns, 1):
    plt.subplot(len(quantitative_df.columns) // 3 + 1, 3, i)
    sns.histplot(quantitative_df[column].dropna(), kde=True, bins=20, color='skyblue', edgecolor='black')
    plt.title(f'Distribution of {column} (NaNs dropped)')
    plt.tight_layout()

plt.show()


In [ ]:
desc = pd.read_excel("/kaggle/input/widsdatathon2025/Data Dictionary.xlsx")
desc[desc['DataType'] == 'Quantitative']


In [ ]:
# Compute correlation matrix excluding 'Id' column
correlation_matrix = quantitative_df.corr()

# Show the top-left part of the matrix
correlation_matrix

In [ ]:
# Drop 'Id' and calculate correlation matrix
import numpy as np

# Filter the upper triangle to avoid duplicate pairs and self-correlation
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find features with correlation > 0.7
high_corr_pairs = upper.stack().reset_index()
high_corr_pairs.columns = ['Feature1', 'Feature2', 'Correlation']
high_corr_pairs = high_corr_pairs[(high_corr_pairs['Correlation'] > 0.7) | (high_corr_pairs['Correlation'] < -0.7)]

print(high_corr_pairs)


In [ ]:
# Calculate missing values and their percentage
missing_values = quantitative_df.isnull().sum()
missing_percentage = (missing_values / len(quantitative_df)) * 100

# Combine both the count and percentage into a single DataFrame
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})

# Display the result
missing_data


In [ ]:
import pandas as pd

# Assuming 'quantitative_df' is your dataframe
mri_column = 'MRI_Track_Age_at_Scan'

# Check if for all rows where 'MRI_Track_Age_at_Scan' is missing, no other column is missing
hypothesis_check = quantitative_df[mri_column].isnull() & quantitative_df.drop(columns=[mri_column]).isnull().any(axis=1)

# If the hypothesis is true, it should be False for all rows (i.e., there should be no rows where MRI is missing but other columns are also missing)
hypothesis_true = not hypothesis_check.any()

# Output the result
if hypothesis_true:
    print("The hypothesis is TRUE: All rows where MRI is missing have no other missing values.")
else:
    print("The hypothesis is FALSE: There are rows where MRI is missing and other columns have missing values.")


In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Test different values of k
results = []
quantitative_df_copy = quantitative_df.copy()
quantitative_df_old=quantitative_df.copy()
for k in range(1, 21):  # Try k = 1 to 20
    # Copy the scaled data again for each loop
    test_data = quantitative_df_copy[quantitative_df_copy['MRI_Track_Age_at_Scan'].notnull()].copy()

    # Randomly mask 10% of values in 'MRI_Track_Age_at_Scan'
    np.random.seed(42)  # for reproducibility
    mask_indices = test_data.sample(frac=0.1).index
    true_values = test_data.loc[mask_indices, 'MRI_Track_Age_at_Scan'].copy()
    test_data.loc[mask_indices, 'MRI_Track_Age_at_Scan'] = np.nan

    # Apply KNN Imputer with current k
    knn_imputer = KNNImputer(n_neighbors=k)
    imputed_array = knn_imputer.fit_transform(test_data)

    # Extract imputed values
    row_positions = test_data.index.get_indexer(mask_indices)
    col_position = test_data.columns.get_loc('MRI_Track_Age_at_Scan')
    imputed_values = imputed_array[row_positions, col_position]

    # Calculate errors
    mae = mean_absolute_error(true_values, imputed_values)
    rmse = np.sqrt(mean_squared_error(true_values, imputed_values))

    results.append((k, mae, rmse))

# Convert results to DataFrame for easy analysis
results_df = pd.DataFrame(results, columns=['k', 'MAE', 'RMSE'])
print(results_df)
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(results_df['k'], results_df['MAE'], marker='o', label='MAE')
plt.plot(results_df['k'], results_df['RMSE'], marker='x', label='RMSE')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Error')
plt.title('KNN Imputation Error vs k')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
# Step 1: Create a copy of the dataframe to avoid modifying the original directly
quantitative_df_copy = quantitative_df.copy()
quantitative_df_old=quantitative_df.copy()
# Step 1: Create a copy of the dataframe to avoid modifying the original directly
quantitative_df_copy = quantitative_df.copy()
quantitative_df_old=quantitative_df.copy()
# Step 2: Scale the data before applying KNN imputation
scaler = StandardScaler()

# Select columns for scaling, assuming you want to scale all columns except 'MRI_Track_Age_at_Scan'
columns_to_scale = quantitative_df_copy.drop(columns=['MRI_Track_Age_at_Scan']).columns

# Apply scaling
quantitative_df_copy[columns_to_scale] = scaler.fit_transform(quantitative_df_copy[columns_to_scale])

# Step 3: Initialize KNN Imputer
knn_imputer = KNNImputer(n_neighbors=9)

# Apply KNN imputation to 'MRI_Track_Age_at_Scan'
quantitative_df_copy['MRI_Track_Age_at_Scan'] = knn_imputer.fit_transform(quantitative_df_copy[['MRI_Track_Age_at_Scan']])
quantitative_df['MRI_Track_Age_at_Scan'] =quantitative_df_copy['MRI_Track_Age_at_Scan'] 
# Step 4: After KNN, replace all remaining missing values in the entire dataframe with the median
quantitative_df.fillna(quantitative_df_copy.median(), inplace=True)

# Step 5: Check the missing values after imputation
missing_values_after_imputation = quantitative_df.isnull().sum()
print(f"Missing values after KNN and median imputation:\n{missing_values_after_imputation}")


# Now 'quantitative_df_temp' has imputed MRI values and median-filled data


In [ ]:
quantitative_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Set up the matplotlib figure
plt.figure(figsize=(20, 15))

# Loop through each column and plot its distribution, dropping NaNs for each column
for i, column in enumerate(quantitative_df.columns, 1):
    plt.subplot(len(quantitative_df.columns) // 3 + 1, 3, i)
    sns.histplot(quantitative_df[column], kde=True, bins=20, color='skyblue', edgecolor='black')
    plt.title(f'Distribution of {column}')
    plt.tight_layout()

plt.show()


In [ ]:
import pandas as pd

# Assuming quantitative_df is your DataFrame
# Calculate Q1 (25th percentile), Q3 (75th percentile) and IQR (Interquartile Range)
Q1 = quantitative_df.quantile(0.25)
Q3 = quantitative_df.quantile(0.75)
IQR = Q3 - Q1

# Calculate outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Detect outliers
outliers = (quantitative_df < lower_bound) | (quantitative_df > upper_bound)

# Count number of outliers and percentage for each column
outlier_counts = outliers.sum()
outlier_percentage = (outlier_counts / len(quantitative_df)) * 100

# Create a DataFrame to combine outlier counts and percentages
outlier_summary = pd.DataFrame({
    'Outlier Count': outlier_counts,
    'Outlier Percentage (%)': outlier_percentage
})

# Print the combined result
print("Outlier Summary (Count and Percentage):")
print(outlier_summary)

In [ ]:
from scipy import stats

# Loop through each column in the dataframe
for column in quantitative_df.select_dtypes(include=['float64', 'int64']).columns:
    result = stats.anderson(quantitative_df[column].dropna(), dist='norm')  # Drop NaN values before applying
    print(f"\nAnderson-Darling Test for {column}:")
    print(f"Statistic: {result.statistic}")
    for i in range(len(result.critical_values)):
        print(f"Critical value at {result.significance_level[i]}% significance level: {result.critical_values[i]}")


In [ ]:
categorical_df.set_index('participant_id', inplace=True)

categorical_df

In [ ]:
desc[desc['DataType'] == 'Categorical ']


In [ ]:
# Compute correlation matrix excluding 'Id' column
correlation_matrix = categorical_df.corr()

# Show the top-left part of the matrix
correlation_matrix

In [ ]:
# Drop 'Id' and calculate correlation matrix
import numpy as np

# Filter the upper triangle to avoid duplicate pairs and self-correlation
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find features with correlation > 0.7
high_corr_pairs = upper.stack().reset_index()
high_corr_pairs.columns = ['Feature1', 'Feature2', 'Correlation']
high_corr_pairs = high_corr_pairs[(high_corr_pairs['Correlation'] > 0.7) | (high_corr_pairs['Correlation'] < -0.7)]

print(high_corr_pairs)


In [ ]:
# Calculate missing values and their percentage
missing_values = categorical_df.isnull().sum()
missing_percentage = (missing_values / len(categorical_df)) * 100

# Combine both the count and percentage into a single DataFrame
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})

# Display the result
missing_data


In [ ]:
import pandas as pd

# Assuming your DataFrame is named df

# Define the columns that should be imputed with -1
columns_to_impute_with_minus_1 = ['Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ']

# Impute the columns with -1
categorical_df[columns_to_impute_with_minus_1] = categorical_df[columns_to_impute_with_minus_1].fillna(-1)

# Impute other columns with the mode (most frequent value)
columns_with_missing = categorical_df.columns[categorical_df.isnull().any()]
for column in columns_with_missing:
    if column not in columns_to_impute_with_minus_1:  # Skip the ones already imputed with -1
        mode_value = categorical_df[column].mode()[0]
        categorical_df[column] = categorical_df[column].fillna(mode_value)

# Now, df has missing values handled as specified


In [ ]:
# Step 5: Check the missing values after imputation
missing_values_after_imputation = categorical_df.isnull().sum()
print(f"Missing values after KNN and median imputation:\n{missing_values_after_imputation}")



In [ ]:
# Get the number of unique values for each column
unique_counts = categorical_df.nunique()
print(unique_counts)
# Calculate the lowest percentage for each column
lowest_percentage = {}

for col in categorical_df.columns:
    # Get value counts and calculate the percentage
    value_counts = categorical_df[col].value_counts(normalize=True) * 100
    
    # Get the lowest percentage
    lowest_percentage[col] = value_counts.min()

# Print the lowest percentage for each column
print(lowest_percentage)
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt

# Set up the number of columns and rows in the grid
n_cols = 3  # Number of columns in the grid
n_rows = 3  # Number of rows in the grid

# Create a figure with subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 12))

# Flatten axes for easy iteration
axes = axes.flatten()

# Loop through each column and plot value counts
for i, col in enumerate(categorical_df.columns[:9]):  # Limit to first 9 columns
    categorical_df[col].value_counts().plot(kind='bar', color='skyblue', ax=axes[i])
    axes[i].set_title(f'Value Counts for {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')
    axes[i].tick_params(axis='x', rotation=45)  # Rotate x-axis labels for better readability

# Adjust layout to avoid overlap
plt.tight_layout()
plt.show()


In [ ]:
# Convert numeric columns to string (if they are represented as numbers but should be categorical)
categorical_df = categorical_df.applymap(str)

# One-hot encode all categorical columns
encoded_df = pd.get_dummies(categorical_df)

# Display the new DataFrame with one-hot encoded columns
encoded_df

# Optionally, you can save the new DataFrame to a CSV file


In [ ]:
fnc_df

In [ ]:
fnc_df.set_index('participant_id', inplace=True)

fnc_df

In [ ]:
# Compute correlation matrix excluding 'Id' column
correlation_matrix = fnc_df.corr()

# Show the top-left part of the matrix
correlation_matrix

In [ ]:
# Drop 'Id' and calculate correlation matrix
import numpy as np

# Filter the upper triangle to avoid duplicate pairs and self-correlation
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find features with correlation > 0.7
high_corr_pairs = upper.stack().reset_index()
high_corr_pairs.columns = ['Feature1', 'Feature2', 'Correlation']
high_corr_pairs = high_corr_pairs[(high_corr_pairs['Correlation'] > 0.7) | (high_corr_pairs['Correlation'] < -0.7)]

print(high_corr_pairs)


In [ ]:
# Calculate missing values and their percentage
missing_values = fnc_df.isnull().sum()
missing_percentage = (missing_values / len(fnc_df)) * 100

# Combine both the count and percentage into a single DataFrame
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})

# Display the result
missing_data
missing_data.sum()



In [ ]:
import pandas as pd

# Assuming quantitative_df is your DataFrame
# Calculate Q1 (25th percentile), Q3 (75th percentile) and IQR (Interquartile Range)
Q1 = fnc_df.quantile(0.25)
Q3 = fnc_df.quantile(0.75)
IQR = Q3 - Q1

# Calculate outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Detect outliers
outliers = (fnc_df < lower_bound) | (fnc_df > upper_bound)

# Count number of outliers and percentage for each column
outlier_counts = outliers.sum()
outlier_percentage = (outlier_counts / len(fnc_df)) * 100

# Create a DataFrame to combine outlier counts and percentages
outlier_summary = pd.DataFrame({
    'Outlier Count': outlier_counts,
    'Outlier Percentage (%)': outlier_percentage
})

# Print the combined result
print("Outlier Summary (Count and Percentage):")
print(outlier_summary)
print(outlier_summary.max())


In [ ]:
# Combine quantitative_df, fnc_df, and encoded_df along columns (axis=1)
combined_df = pd.concat([quantitative_df, fnc_df, encoded_df], axis=1)

# Check the first few rows of the combined DataFrame
combined_df


In [ ]:
# Compute correlation matrix excluding 'Id' column
correlation_matrix_t = combined_df.corr()

# Show the top-left part of the matrix
correlation_matrix_t

In [ ]:
# Drop 'Id' and calculate correlation matrix
import numpy as np

# Filter the upper triangle to avoid duplicate pairs and self-correlation
upper = correlation_matrix_t.where(np.triu(np.ones(correlation_matrix_t.shape), k=1).astype(bool))

# Find features with correlation > 0.7
high_corr_pairs = upper.stack().reset_index()
high_corr_pairs.columns = ['Feature1', 'Feature2', 'Correlation']
high_corr_pairs = high_corr_pairs[(high_corr_pairs['Correlation'] >= 0.75) | (high_corr_pairs['Correlation'] <= -0.75)]

print(high_corr_pairs)


In [ ]:
solutions_df